# Determine the 2D Basis of a Plane

The first step in simulating interfaces is the determination of the two-dimensional periodicity (i.e. the basis vectors) of the plane. The interfaces are two-dimensional sections of the underlying three-dimensional lattice, and hence, the interface will exhibit the periodicity of the corresponding 2D lattice of the plane. The technique to determine the basis vectors is outlined in the following article:

[**An efficient algorithm for computing the primitive bases of a general lattice plane.**](https://scripts.iucr.org/cgi-bin/paper?rg5087)
Journal of Applied Crystallography
Banadaki, A. D., & Patala, S. (2015). , 48(2), 585-588.

In this tutorial, we will discuss the steps involved in determining the basis vectors of the 2D plane using the **byxtal** package. Please follow the installation steps (link needed!!) to acquire the byxtal package and import all the required packages that we need for completing this tutorial.

## Miller Indices and Conventions:

1. Miller Indices are often used to refer to a given crystallographic plane in crystals. 
2. However, various conventions are commonly used in determining the Miller Indices that can change the indices of the plane. For example, in the FCC lattice, one could either use the cubic unit-cell or the primitive cell to index planes and directions, resulting in completely different indices for the same plane. Therefore, we would like to declare our conventions in defining the Miller Indices to avoid potential confusion. 
3. By definition, the Miller indices of a plane, denoted by $(h \, k \, l)$, refer to the indices of the lattice vector perpendicular to the plane expressed in the reciprocal lattice. Therefore, the indices will depend on the reference lattice used (e.g. the cubic unit-cell or the primitive cell).
4. In the **byxtal** package, we perform the calculations in the primitve cell. The reason simply is that, in the primitive lattice, all the lattice points are expressed using integers. This helps with some of the algebraic manipulations that are preformed in the package.
5. **Unless otherwise specified, the indices in byxtal package are in reference to the primitve cell.**
6. In the present tutorial, we also discuss how to convert the indices from one reference frame to the other (e.g. from the primitive cell to the cubic unit-cell).


[//]: # "Miller Indices are defined as the reciprocal of intercepts of a crystallographic plane with the Unit Cell Basis Vectors. The confusion is often caused by the definition of the Unit Cell. In f.c.c and b.c.c lattices primitive basis vectors are non-orthogonal while the supercell basis vectors are orthogonal. Most importantly since the reciprocal and direct lattices are the identical in f.c.c and b.c.c, the Miller Indices (defined in supercell unit cell) are the same as the plane normal indices. This unique property and convenience of using an orthogonal set of basis vectors is the root cause of the bipartisan approach to defining the Miller Indices. The downfall of such an approach is the fact that it does not have much utility in other types of lattices e.g. hcp. Therefore in GBpy whenever we use the term Miller Indices, we are referring to the reciprocals of intercepts of a crystallographic plane with the primitive Basis Vectors."
[//]: # (Miller Indices: reciprocal of intercepts of a crystallographic plane with the primitive Basis Vectors, or alternatively, Miller Indices: normal vector indices of the plane defined in primitive reciprocal lattice. The above interchangeable definitions are consistently used in GBpy for referring to Miller Indices. Other conventions in defining the Miller Indices are ultimately converted to the above definition. In the present tutorial we manually extract various Indices and demonstrate how they can be fed to the GBpy. In practice such conversions are not necessary as long as the user is consistent with the definitions of GBpy, and can be used for verifying the answer.)

Let's start with importing the **byxtal** package and other modules that we will use in this tutorial.

In [1]:
import sys
# sys.path.insert(0, './../../../../gbpy/')
import numpy as np
from sympy.matrices import Matrix, eye, zeros;

## Problem Definition:

In the tutorial, we will determine the planar basis of a crystallographic plane in the FCC lattice. Consider the plane whose normal vector is along the direction $[2 3 1]$ expressed in the cubic unit-cell reference frame. Since the normal vector to the plane is provided in the orthogonal unit-cell basis, we will first determine the Miller Indices of the plane (using the primitive cell bases).

### Finding Miller Indices:

A vector in the space can be expressed in any basis, of course with varying components. Vector $\vec{v}$ in basis A can be expressed as:

\begin{equation}
\vec{v} = \mathcal{B}_A v_A
\end{equation}

Similarly we can define the plane normal $\vec{n}$ in any basis. For instance we can define $\vec{n}$ in unit-cell basis ($\mathcal{B}_{PO}$) or in primitive-cell basis $\mathcal{B}_P$; we can write:

\begin{equation}
\vec{n} = \mathcal{B}_{PO} n_{PO} = \mathcal{B}_{P} n_{P}
\end{equation}

The conversion from one basis to the other can be determined by using the components of the basis vectors of one of the frames (e.g. $P$) in the other frame (e.g. $PO$):

\begin{equation}
\mathcal{B}_{P} = \mathcal{B}_{PO} \Lambda_{P}^{PO}
\end{equation}

where, $\Lambda_P^{PO}$ is a $3 \times 3$ matrix with its columns representing the components of basis vectors of $P$ frame in the $PO$ basis. For example, for an FCC lattice, $\Lambda_P^{PO}$ is given below.

In [2]:
l_p_po = 1.0 * Matrix([[0.,0.5,0.5],[0.5,0.,0.5],[0.5,0.5,0.]])
l_p_po

Matrix([
[  0, 0.5, 0.5],
[0.5,   0, 0.5],
[0.5, 0.5,   0]])

We can now determine the components of the vector $\vec{n}$ in the $P$ reference frame as follows:

\begin{align}
\mathcal{B}_{P} n_{P} &= \mathcal{B}_{PO} n_{P0} \\ \nonumber
\mathcal{B}_{PO} \Lambda_{P}^{PO} n_{P} &= \mathcal{B}_{PO} n_{P0} \\ \nonumber
\Lambda_{P}^{PO} n_{P} &= \mathcal{B}_{PO} n_{P0} \\ \nonumber
n_{P} &= \Lambda_{PO}^{P} n_{P0}
\end{align}

where $\Lambda_{P}^{PO} = \left( \Lambda_{PO}^{P} \right)^{-1}$.


1. **To determine the Miller indices**, we have to express the components of the normal vector $\vec{n}$ in the reference frame of the reciprocal lattice (the reciprocal of the primitive cell).

2. The basis vectors of the reciprocal of the primitve lattice are denoted using the symbol $\mathcal{B}^*_{P}$, and are given in the $PO$ reference frame as:

\begin{equation}
\mathcal{B}^*_{P} = \mathcal{B}_{PO} \Lambda_{P*}^{PO}
\end{equation}

3. $\Lambda_{P*}^{PO}$ can be computed using the byxtal package using the function `bxt.find_csl_dsc.reciprocal_mat()`. For the sake of convenience we abbreviate the imported module `bxt.find_csl_dsc` as `fcd`. The code is shown below.

In [3]:
import gbpy.byxtal.find_csl_dsc as fcd
l_rp_po = fcd.reciprocal_mat(l_p_po)
l_rp_po

TypeError: No loop matching the specified signature and casting was found for ufunc det

where we use the variable `l_rp_po` to represent $\Lambda_{P*}^{PO}$. Now, we can determine the indices of $\vec{n}$ in the $P^*$ reference frame, using equation (4) as:

\begin{equation}
n_{P^*} = \Lambda_{PO}^{P*} n_{P0}
\end{equation}

Use the following code to determine the components $n_{P^*}$:

In [4]:
l_po_rp = (l_rp_po).inv()
n_po = Matrix([[2], [3], [1]])
n_rp = l_po_rp*n_po
n_rp

Matrix([
[2.0],
[1.5],
[2.5]])

Remember, that the Miller Indices ought to be integers (without common factors). We have to find a common scaling factor for all the components such that the result is going to be scalar. We have implemented a function named `int_finder` that performs this task for a variety of input types (e.g. rows and columns of matrices). For irrational numbers int_finder accepts a tolerance and performs the same operation on the closest rational number within the specified tolerance. You can find this function in the package as: `byxtal.integer_manipulations.int_finder()`. Therefore, we repeat the previous steps and pass the results to the `int_finder` function to obtain the integer Miller indices.

In [3]:
import byxtal.integer_manipulations as iman
ni_rp = iman.int_finder(n_rp)
ni_rp

NameError: name 'n_rp' is not defined

## Finding the Planar Basis:

1. From the previous section, we found the Miller Indices of an FCC plane with the normal along $n_{PO} = [2 3 1]$ to be $(4 3 5)$. 

2. Now all we have to do is to pass the obtained indices to `bp_basis`, which is a function that gets the Miller Indices (expressed using the primitive cell) as the input and returns a $3 \times 2$ matrix, where the columns represent the components of the basis vectors in the primitive $(P)$ reference frame.

3. Also the obtained vectors are in the [reduced form](https://en.wikipedia.org/wiki/Lenstra%E2%80%93Lenstra%E2%80%93Lov%C3%A1sz_lattice_basis_reduction_algorithm). You can find the bp_basis function in the following path: `byxtal.bp_basis.bp_basis()`. To find the basis vector of a plane with the Miller Indices of $(4 3 5)$ use the following syntax:

In [8]:
import byxtal.bp_basis as bpb
l_2D_p = Matrix(bpb.bp_basis(ni_rp))
l_2D_p

NameError: name 'ni_rp' is not defined

To express the obtained basis in the orthogonal basis (i.e. supercell f.c.c) one needs to perform the following conversion of bases:

\begin{equation}
\Lambda_{2D}^{PO} = \Lambda_{P}^{PO} \times \Lambda_{2D}^{P}
\end{equation}


In [7]:
l_2D_po = l_p_po*l_2D_p
l_2D_po

Matrix([
[-1.0,    0],
[ 0.5, -0.5],
[ 0.5,  1.5]])

## Summary

1. At the interface of a bicrystal, the $\Lambda_{2D}^{PO}$ provides a basis for the interface. 
2. If the two crystals are related to each other by a $\Sigma$-rotation, the obtained $\Lambda_{2D}^{po}$ is the two-dimensional basis for the two-dimensional coincidence site lattice at the interface. Therefore, the bicrystal conserves its periodicity in the obtained 2D-basis. 
3. In other words the obtained basis is in fact the basis for the unit cell of the bicrystal and since it is in the reduced form, it is going to have the least skewness, hence ideal for constructing a periodic simulation box.

The above process is frquently repeated for simulation of grain boundaries. Therefore, we have developed a set of functions that make the conversion of indices more convenient and will accept various conventions for the Miller Indices. Please refer to the grain boundary 2D-CSL tutorial for how to use these functions.